### Summary

Refactor the Linear_solvers package, so that the quantum algorithm (HHL) accepts oracles for the different blocks and computes de different parameters according to the error bounds calculated in [1].

### Workflow

An instance of the algorithm can be created by specifying the error tolerance, the problem to be solved is only specified when the `solve()` method is called:
```python
from qiskit.aqua.algorithms.linear_solvers.hhl import HHL

# initialise the algorithm. HHL() defaults to epsilon=1e-2
hhl = HHL(epsilon=1e-3)
```

Alternatively, it is possible to specify the different parameters of HHL (i.e. the number of qubits to represent the eigenvalues, the evolution time, ...) through setter methods. However it should be noted that in this case the error tolerance won't be guaranteed anymore. The reason for this is because from the theory in [1] we can calculate the different parameters to achieve an overall accuracy `epsilon`, but once one of these parameters is changed by the user, `epsilon` will change as well.

```python
hhl.n_l = 5 # number of qubits to represent the eigenvalues. This line will prompt a warning regarding epsilon
```

The `solve()` method accepts three arguments: the matrix defining the linear system, the vector right hand side of the equation (i.e. writing $Ax=b$, respectively $A$ and $b$), and the (list of) observable(s) to be computed out of the solution vector $x$. The latter is required because the output of the quantum algorithm is a quantum state representing the vector $x$, however to learn all the components of this vector would take a linear time in its dimension diminishing any speedup obtained by the quantum algorithm. Thus, we can only compute functions from $x$ (the so called observables) to learn information about the solution.

#### Input matrix

There are several ways to specify $A$. Intuitively, it can be specified as a numpy array:
```python
matrix = [[1/2, -2], [-2, 1/2]]
```
If we know how to implement $e^{iAt}$, we can also give this circuit as an input. The circuit needs to be able to accept $t$ as a parameter, since this parameter will be specified within HHL. 

```python
from qiskit.circuit.library.blueprintcircuit import BlueprintCircuit
class MatrixCircuit(BlueprintCircuit):
    def __init__(self, num_state_qubits, name='mcirc'):
        super().__init__(name=name)
        self.num_state_qubits = num_state_qubits
        
    ...
    
    def _build(self, evo_time=1):
        # build code
        
matrix = MatrixCircuit(1)
```

The matrix can also be specified as an operator
```python
matrix = (-1.05 * I) + (0.39 * Z) + (0.18 *  X) 

```

#### Input vector

HHL also accepts different types for the vector. Again, it can be specified as a numpy array:
```python
vector = [1, 0]
```
Or as a circuit
```python
from qiskit import QuantumCircuit

vector = QuantumCircuit(1)
vector.h(0)
```

#### Input observable

There are several types of available observables which can be given as input:
```python
from qiskit.aqua.algorithms.linear_solvers.observables import AbsoluteAverage, ExpectedValue,MatrixFunctional

observables = [AbsoluteAverage(), ExpectedValue()]
```
The `MatrixFunctional` class takes a matrix for its constuctor method, which can also be specified as a numpy array.
```python
observables.append(MatrixFunctional([[1,2],[2,1]]))
```

Once these variables are specified, we can run the algorithm and obtain the calculated value for each observable:
However some observables will depend on the dimension of the system, which ideally one only specifies once `hhl.solve` is called. Therefore, we could allow to add observables from this method as well.
```python
# run the algorithm and add an observable which depends on the dimension 
result = hhl.solve(matrix=matrix, vector=vector, observable=observables)

# array with the calculated values for each observable
observable_values = result.value

```

Finally, some algorithms can calculate the Euclidean norm of the vector solution. This can be accessed via
```python
result.euclidean_norm
```
which returns either a `float` or `None`.

Another option is to run the `LinearSystem` algorithm without specifying an observable and then use the circuit as the input for the observable class:
```python
hhl = HHL(epsilon=1e-3)
circuit = hhl.construct_circuit(matrix, vector)

observable = AbsoluteAverage()

result = observable.evaluate(circuit)
```

### Future work

- Add the Richardson extrapolation logic.
- Add a `StateObservable` which returns the full vector solution (e.g. tomography).

### References
[1] Carrera Vazquez, A., Hiptmair, R., & Woerner, S. (2020). Enhancing the Quantum Linear Systems Algorithm using Richardson Extrapolation. `arXiv:2009.04484 <http://arxiv.org/abs/2009.04484>`